https://youtu.be/ozCpmkbJNJE?si=z4RrHJJ6Y6iDlx-6 - set the env 

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 19:10:11--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 19:10:11 (41.8 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [1]:
import minsearch

In [2]:
import json

In [3]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = [] 
for course_dict in docs_raw: 
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)
        

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = "the course has already started, can I still enroll?"


In [8]:
index.fit(documents)

In [9]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    query = q, 
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict = boost,
    num_results=5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI(base_url="https://api.perplexity.ai")

In [13]:
response = client.chat.completions.create(
    model = "llama-3-sonar-small-32k-chat",
    messages = [{"role": "user", "content": q}]
)

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT doesn't contain the answer, output NONE. 

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
prompt = build_prompt(q, results)

response = client.chat.completions.create(
    model = "llama-3-sonar-small-32k-chat",
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Based on the provided CONTEXT, the answer to the QUESTION "the course has already started, can I still enroll?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks.'

In [16]:
def llm(prompt):
    
    response = client.chat.completions.create(
    model = "llama-3-sonar-small-32k-chat",
    messages = [{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results


def rag(query):
    search_results = search(query) #toy search engine 
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
    

In [17]:
query = "the course has already started, can I still enroll?"

rag(query)


'Based on the CONTEXT, the answer to the QUESTION "the course has already started, can I still enroll?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks.'

#### Modify Search Engine 
Running ElasticSearch:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [18]:
from elasticsearch import Elasticsearch


In [19]:
es_client = Elasticsearch("http://localhost:9200")

In [20]:
es_client.info()

ObjectApiResponse({'name': 'ad9e11c74b78', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5mtWfk0JQt2akrJXCY21SA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name  = "course-questions"

es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [22]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc )

100%|████████████████████████████████████████████| 1010/1010 [00:25<00:00, 40.39it/s]


"question^3", "text", "section" =>  question is 3 times more important;
"size": 5 => consider only 5 answers from the database 


In [24]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    # iterate through the internal structure 
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [26]:
def rag(query):
    search_results = elastic_search(query)  
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)

"ANSWER: Yes, even if you don't register, you're still eligible to submit the homeworks."

In [34]:

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']
       

48.077904